In [1]:
import sys
import time
import pickle
import gzip
from random import randint
from scipy import misc
import scipy.ndimage
from scipy.sparse import csc_matrix, issparse
import numpy as np
import datetime as dt
from sklearn.cluster import KMeans,MiniBatchKMeans
import matplotlib.pyplot as plt
import json
import csv
import collections
import math
import sys

#setting path
DATA_PATH = 'data/mnist/'

IMAGES_TRAIN = 'data_training'
IMAGES_TEST = 'data_testing'

RANDOM_SEED = 42
N_CLASSES = 10
N_FEATURES = 28 * 28

#import data+label
data_training = DATA_PATH+IMAGES_TRAIN
data_testing = DATA_PATH+IMAGES_TEST
ft = gzip.open(data_training, 'rb')
TRAINING = pickle.load(ft)
ft.close()
ft = gzip.open(data_testing, 'rb')
TESTING = pickle.load(ft)
ft.close()

In [121]:
import logging

In [122]:
logNN = logging.getLogger('NN')
logNN.propagate = False
logNN.setLevel(logging.DEBUG)

fh1 = logging.FileHandler('log/nn.log')
fh1.setLevel(logging.DEBUG)

ch1 = logging.StreamHandler()
ch1.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
fh1.setFormatter(formatter)
ch1.setFormatter(formatter)

logNN.addHandler(fh1)
logNN.addHandler(ch1)

In [123]:
def ReLU (x, derivate = False):
    if not derivate:
        return x * (x > 0)
    else:
        return 1 * (x > 0)
    
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [136]:
class NN:
    def __init__(self, neurons, training, testing, lr, mu, minibatch=None):
        self.numEx, self.numTest = 60000, 10000
        self.neurons=neurons
        self.training=training
        self.testing=testing
        self.lr = lr
        self.mu = mu
        self.minibatch=minibatch
        
        #initalizate He at al.
        self.Wh = np.random.randn(N_FEATURES, self.neurons) * math.sqrt(2.0/self.numEx)
        self.bh = np.random.randn(1, self.neurons) * math.sqrt(2.0/self.numEx)
        self.Wo = np.random.randn(self.neurons, 10) * math.sqrt(2.0/self.numEx)
        self.bo = np.random.randn(1, 10) * math.sqrt(2.0/self.numEx)
        
        self.target_train = self.training[1][0:self.numEx]
        self.target_test = self.testing[1][0:self.numTest]
        
        self.targetForUpd=np.zeros((self.numEx, 10), dtype=int)
        for i in range(self.numEx):
            self.targetForUpd[i,training[1][i]] = 1
     
        self.epoch = 0
        self.vh = 0
        self.vo = 0 
        self.vbh = 0
        self.vbo = 0
        
    def predict(self, X):
        outputs=[]
        H=ReLU(np.dot(X, self.Wh)+self.bh)
        outputs.append(H)
        outputs.append(sigmoid((np.dot(H, self.Wo))+self.bo))
        return outputs
        
    def predictHotClass(self, X):
        return np.argmax(self.predict(X)[-1], axis=1).reshape(-1,1)
    
    def accuracy(self, X, t):
        lengthX = X.shape[0]
        correct = 0
        predictons = self.predictHotClass(X)
        for i in range(lengthX): 
            if predictons[i] == t[i]:     
                correct+=1
        return np.round(correct/lengthX*100, 2)
        
    def updateMomentum(self, X, t, nEpochs, learningRate, momentumUpdate):
        numBatch = (int)(self.numEx/self.minibatch)
        lr = 0.0001 + learningRate * math.e ** -(self.epoch / nEpochs)
       
        for nb in range(numBatch):
            indexLow = nb*self.minibatch
            indexHigh = (nb+1)*self.minibatch
            hidden, y = self.predict(X[indexLow:indexHigh])        
            delta_y = y * (1-y) * (y-t[indexLow:indexHigh])       
            delta_hidden = np.dot(delta_y, self.Wo.T) * ReLU(hidden, derivate=True)   

            delta_h = - lr * np.dot(X[indexLow:indexHigh].T, delta_hidden) #X[indexLow:indexHigh].T
            delta_o = - lr * np.dot(hidden.T, delta_y)
            delta_bo = - lr * np.sum(delta_y, axis=0, keepdims=True)
            delta_bh = - lr * np.sum(delta_hidden, axis=0, keepdims=True)

            self.Wh += delta_h + momentumUpdate * self.vh
            self.bh += delta_bh + momentumUpdate * self.vbh
            self.Wo += delta_o + momentumUpdate * self.vo  
            self.bo += delta_bo + momentumUpdate * self.vbo

            
            self.vh = delta_h
            self.vbh = delta_bh
            self.vo = delta_o  
            self.vbo = delta_bo
        

    def updateMiniBatch(self, X, t, learningRate):        
        numBatch = (int)(self.numEx/self.minibatch)
        lr = 0.0001 + learningRate * math.e ** -(self.epoch/400 )
        
        for nb in range(numBatch):
            indexLow = nb*self.minibatch
            indexHigh = (nb+1)*self.minibatch
            hidden, y = self.predict(X[indexLow:indexHigh])   
            delta_y = y * (1-y) * (y-t[indexLow:indexHigh])       
            delta_hidden = np.dot(delta_y, self.Wo.T) * ReLU(hidden, derivate=True)   

            delta_h = - lr * np.dot(X[indexLow:indexHigh].T, delta_hidden)
            delta_o = - lr * np.dot(hidden.T, delta_y)
            delta_bo = - lr * np.sum(delta_y, axis=0, keepdims=True)
            delta_bh = - lr * np.sum(delta_hidden, axis=0, keepdims=True)

            self.Wh += delta_h
            self.bh += delta_bh
            self.Wo += delta_o  
            self.bo += delta_bo
        
        
    def ETAepoch(self,start_time):
        diff = dt.datetime.now() - self.start_time
        eta = divmod(diff.days * 86400 + diff.seconds, 60)
        if eta[0] != 0:
            ret = str(eta[0])+"m"
        else:
            ret = ""
        ret += str(eta[1])+"s"
        return ret
        
    def print_message_iter(self,iteration,accu_train,eta):
        len_eta = len(eta)
        space_fill = 6 - len_eta
        eta = "("+eta+")"
        for _ in range(space_fill):
            eta += " "
        message = 'Epoch '+str(self.epoch).zfill(3) + " "+eta+" "
        message += 'Accuracy: '+str(accu_train).zfill(4)+'%\t'
        return message
        
    def train(self):
        train=self.training[0][0:self.numEx]
        test=self.testing[0][0:self.numTest]
        num_epochs = 400
       
        
        self.start_time = dt.datetime.now()
        #print(self.start_time)
        logNN.info("neurons= "+str(self.neurons)+" learning rate="+str(self.lr)+ " momentum update=" +str(self.mu)+" minibatch="+str(self.minibatch))
        #logNN.info("learning rate="+str(self.lr)+" Vanilla minibatch(100)")
        while self.epoch <= num_epochs:
            #self.updateMiniBatch(train, self.targetForUpd, self.lr)
            self.updateMomentum(train, self.targetForUpd, num_epochs, self.lr, self.mu)
            if self.epoch % 100 == 0:
                logNN.info("epoch: "+str(self.epoch)+":  "+str(self.accuracy(train, self.target_train)))
            
            if self.epoch % 25 == 0:
                logNN.debug("epoch: "+str(self.epoch)+":  "+str(self.accuracy(train, self.target_train)))
            
            self.epoch += 1  
                        
        logNN.info("Accuracy on Test: "+str(self.accuracy(test, self.target_test)))
        logNN.debug("---------------------------------------------------------------------")
        
    def getWeigth(self):
        return self.Wh, self.Wo
       

In [137]:
nn = NN(neurons=100, training=TRAINING, testing=TESTING, lr=0.003, mu=.99, minibatch=100)
nn.train()


2019-05-21 17:12:15,875 - INFO - neurons= 100 learning rate=0.003 momentum update=0.99 minibatch=100
2019-05-21 17:12:19,072 - INFO - epoch: 0:  89.66
2019-05-21 17:15:53,920 - INFO - epoch: 100:  99.42
2019-05-21 17:19:37,873 - INFO - epoch: 200:  99.5
2019-05-21 17:23:08,407 - INFO - epoch: 300:  99.53
2019-05-21 17:26:40,475 - INFO - epoch: 400:  99.55
2019-05-21 17:26:41,629 - INFO - Accuracy on Test: 98.03
